In [2]:
from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from utils import get_subfolders_with_files, is_image

image_dir = "../test-data/"
filenames = list(get_subfolders_with_files(image_dir, is_image, yield_by_one=True))
len(filenames)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['DALI_DISABLE_NVML'] = '1'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
# export DALI_DISABLE_NVML=1
# export DALI_RESTRICT_PINNED_MEM=1
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator

max_batch_size = 8

In [117]:
@pipeline_def
def get_dali_image_pileline(filenames, labels, width, height, mean=None, std=None):
    jpegs, labels = fn.readers.file(files=filenames, labels=labels)
    images = fn.decoders.image(jpegs, device='mixed')
    shape = fn.shapes(images)
    images = fn.resize(images, resize_x=width, resize_y=height)
    if mean is not None or std is not None:
        print(f'Using mean: {mean} std: {std}')
        mean_ = np.array([[mean]]) if mean is not None else None
        std_ = np.array([[std]]) if std is not None else None
        images = fn.normalize(images, mean=mean_, stddev=std_)
    images = fn.transpose(images, perm=[2, 0, 1])
    return images, labels, shape[0], shape[1]



# def get_dali_dataloader(filename_to_id, model_cfg):
#     pipe = get_dali_image_pileline(
#         filenames=list(filename_to_id.keys()), labels=list(filename_to_id.values()),
#         width=model_cfg.input_shape[0], height=model_cfg.input_shape[1],
#         mean=model_cfg.img_mean, std=model_cfg.img_std,
#         batch_size=model_cfg.batch_size, num_threads=model_cfg.num_workers, device_id=0
#     )
#     pipe.build()
#     return DALIGenericIterator([pipe], ['images', 'filename_map', 'height', 'width'])

batch_size=1
sequence_length=8
initial_prefetch_size=11
shuffle=True
n_iter=6

@pipeline_def
def video_pipe(filename):
    video, labels, start_frame_num = fn.readers.video(device="gpu", filenames=[filename], labels=[], sequence_length=sequence_length,
                                     random_shuffle=False, image_type=types.RGB, dtype=types.FLOAT, initial_fill=initial_prefetch_size, enable_frame_num=True)
    shapes = fn.shapes(video)[1:3]
    video = fn.resize(video, size=[2048, 1024], device="gpu")
    video = fn.crop_mirror_normalize(video, output_layout="FCHW", mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], image_type=types.RGB, device="gpu")
    return video, shapes, start_frame_num


In [118]:
pipe = video_pipe(filename='src_cfr_test.mp4', batch_size=batch_size, num_threads=2, device_id=0, seed=12345)
pipe.build()

dali_iter = DALIGenericIterator([pipe], ['images', 'shapes', 'start_frame_num'])

/usr/local/lib/python3.8/dist-packages/nvidia/dali/fn.py:76: DeprecationWarning: The argument ``image_type`` is no longer used and will be removed in a future release.
  return op_class(**init_args)(*inputs, **call_args)
[/opt/dali/dali/operators/reader/loader/video_loader.h:178] ``file_list_include_preceding_frame`` is set to False (or not set at all). In future releases, the default behavior would be changed to True.


In [122]:
for i, data in enumerate(dali_iter):
    batch = data[0]
    images = batch['images'].squeeze()
    start_frame_num = batch['start_frame_num'].cpu().numpy()
    shapes = batch['shapes'].cpu().numpy()
    img_masks = None
    metadata = []
    for first_frame_num, shape in zip(start_frame_num, shapes):
        for off in range(images.shape[0] // len(shapes)):
            n_frame = first_frame_num.item() + off
            metadata.append({
                "height": shape[0], "width": shape[1], "image_path": f'{n_frame:0>5}.jpg',
            })
    print(images.shape)
    print(metadata)
    break

[[24]]
torch.Size([8, 3, 2048, 1024])
[{'height': 1080, 'width': 1920, 'image_path': '00024.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00025.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00026.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00027.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00028.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00029.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00030.jpg'}, {'height': 1080, 'width': 1920, 'image_path': '00031.jpg'}]


In [43]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

def show_sequence(sequence, label):
    columns = 1
    rows = (sequence_length + 1) // (columns)
    fig = plt.figure(figsize = (32,(16 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns-1):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.suptitle("label " + str(label[0]), fontsize=30)
        plt.imshow(sequence[j])

In [85]:
ITER = 5
for i in range(ITER):
    sequences_out, shapes, start_frame_num = pipe.run()
    sequences_out = sequences_out.as_cpu().as_array()
    shapes = shapes.as_cpu().as_array()
    start_frame_num = start_frame_num.as_cpu().as_array()
    print(sequences_out.shape, shapes.shape, start_frame_num.shape)
    # show_sequence(sequences_out[0], start_frame_num[0])

(2, 1, 3, 2048, 1024) (2, 2) (2, 1)
(2, 1, 3, 2048, 1024) (2, 2) (2, 1)
(2, 1, 3, 2048, 1024) (2, 2) (2, 1)
(2, 1, 3, 2048, 1024) (2, 2) (2, 1)
(2, 1, 3, 2048, 1024) (2, 2) (2, 1)


array([[1080, 1920],
       [1080, 1920]], dtype=int64)

In [ ]:
import numpy as np
np.array([1]).item()
# pipe = simple_pipeline(filenames=filenames[:16], labels=list(range(len(filenames[:16]))), r_width=2048, r_height=1024, batch_size=max_batch_size, num_threads=1, device_id=0)
video_name = '../test-video-fixed-framerate.mp4'
video_name = '/mnt/c/Users/Vlad/Desktop/ny-guiderails/test-video-from-images.mp4'
video_name = '../test.mp4'
pipe = video_pipe(video_name, sequence_length=8, initial_prefetch_size=16, num_threads=1, batch_size=8, device_id=0)
pipe.build()
res = pipe.run()
layout
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

def show_images(image_batch):
    columns = 4
    rows = (max_batch_size + 1) // (columns)
    fig = plt.figure(figsize = (32,(32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.imshow(image_batch.at(j))